<a href="https://colab.research.google.com/github/hourglasshoro/research-notebook/blob/main/tacotron_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!unzip jsut_ver1.1.zip > /dev/null

In [ ]:
!apt install aptitude swig > /dev/null
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null
!pip install mecab-python3==0.996.3 > /dev/null
!pip install pykakasi > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null



Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11.6M  100 11.6M    0     0  1643k      0  0:00:07  0:00:07 --:--:-- 2424k
./../../libexec/iconv_euc_to_utf8.sh ./Noun.verbal.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Conjunction.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Prefix.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.org.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Suffix.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.adjv.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Postp-col.csv 
./../../libexec/iconv_euc_to_utf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import unicodedata
import string
import MeCab
import pykakasi.kakasi as kakasi

def format_text(text):
    text = unicodedata.normalize("NFKC", text)  # 全角記号をざっくり半角へ置換（でも不完全）

    # 記号を消し去るための魔法のテーブル作成
    table = str.maketrans("", "", string.punctuation  + "「」・")
    text = text.translate(table)
    text = text.translate(str.maketrans({'、' : ',', '。' : '.'}))
    return text

def getPronunciation(text):
    m_result = m.parse(text).splitlines() #mecabの解析結果の取得
    m_result = m_result[:-1] #最後の1行は不要な行なので除く

    pro = '' #発音文字列全体を格納する変数
    for v in m_result:
        if '\t' not in v: continue
        surface = v.split('\t')[0] #表層形
        p = v.split('\t')[1].split(',')[-1] #発音を取得したいとき
        #p = v.split('\t')[1].split(',')[-2] #ルビを取得したいとき
        #発音が取得できていないときsurfaceで代用
        if p == '*':
          p = surface
        pro += p

    pro = conv.do(pro) #ひらがなをカタカナに変換
    pro = format_text(pro) #余計な記号を削除

    return pro

In [ ]:
import subprocess

cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0}".format(path))

In [ ]:
kakasi = kakasi()
kakasi.setMode("K","a")
conv = kakasi.getConverter()

In [ ]:
text = '水をマレーシアから買わなくてはならないのです。'
print(text)
print(getPronunciation(text))

水をマレーシアから買わなくてはならないのです。
mizu wo mareeshia kara kawa naku te wa nara nai no desu. 


In [ ]:
baseUrl = '/content/drive/MyDrive/Research/dataset/jsut/jsut_ver1.1'
datasetUrl = '/content/jsut_ver1.1'

In [ ]:
import os

In [ ]:
dirs = ['basic5000','precedent130','repeat500','voiceactress100','loanword128','travel1000','countersuffix26','onomatopee300','utparaphrase512']

In [ ]:
out_path = os.path.join(baseUrl, "filelist.txt")
with open(out_path, mode='w', encoding='utf-8') as fout:
  for dir in dirs:
    file_path = os.path.join(baseUrl,dir,'transcript_utf8.txt')
    f = open(file_path, 'r', encoding='UTF-8')
    datalist = f.readlines()
    for data in datalist:
      data = str.split(data,":")
      wav_file = data[0] + '.wav'
      wav_file = os.path.join(datasetUrl, dir, 'wav' ,wav_file)
      text = getPronunciation(data[1])
      fout.write(wav_file + '|' + text + '\n')

    f.close()

In [ ]:
  file_path = os.path.join(baseUrl, "filelist.txt")
  test_path = os.path.join(baseUrl, "ljs_audio_text_test_filelist.txt")
  train_path = os.path.join(baseUrl, "ljs_audio_text_train_filelist.txt")
  val_path = os.path.join(baseUrl, "ljs_audio_text_val_filelist.txt")
  f = open(file_path, 'r', encoding='UTF-8')
  datalist = f.readlines()
  test_div = 10
  with open(test_path, mode='w') as ftest, open(train_path, mode='w') as ftrain, open(val_path, mode='w') as fval:
    for i, data in enumerate(datalist):
      if i % test_div == 0:
        if i % (test_div * 5) == 0:
          fval.write(data)
        else:
          ftest.write(data)
      else:
        ftrain.write(data)